**TP n°4** : Deep Learning pour le clustering en grande dimension

#Plan

##Partie I:

Rappels sur le clustering:
- k-moyennes , un rapide rappel
- problème non-linéaire - espace de redescription
- les limites de la grande dimension

##Partie II:

- réduire la dimensionnalité avec un auto-encodeur
- séparer linéairement avec un auto-encodeur variationnel


Durée : 2 h

**Partie II** : Partionnement dans l'espace latent d'un autoencodeur

Dans la première partie, on a présenté sur des exemples "naïfs" quelques limites génériques des méthodes de partitionnement. Avec des données images, ces limites sont atteintes : 
- regrouper selon les "contenus" est un problème profondément non-linéaire
- une description de l'image dans un espace de plus petite dimension est possible dans la mesure où le nombre de degrés de liberté des objets contenus par l'image est restreint.

Dans cette partie, on voit comment avec un "auto-encodeur", on peut construire un espace de redescription -l'espace latent- où le partionnement devient facile. Cette possibilité est illustrée sur le jeu de données MNIST.
L'**exercice 1** présente une méthode de réduction de la dimensionnalité par réseaux de neurones: l'auto-encodeur. A partir d'une achitecture simplifiée, nous visualisons la façon dont le signal s'organise au niveau de la "couche latente". Nous vérifions qu'effectivement, des clusters se forment et qu'il correspondent plus ou moins aux différents chiffres. Dans la suite suite, on cherche à améliorer ce résultat. \
Dans l'**exercice 2**, on présente le concept d'auto-encodeur variationnel (vae). Un premier exemple vient illustrer comment, avec un vae, on peut contraindre l'organisation de l'espace latent. \
Dans l'**exercice 3**, un VAE mieux adapté au problème de clustering est présenté. Converti en classifieur (approche transductive), il permet d'atteindre des justesses de plus de 90% sur le jeu de test des chiffres de MNIST sans l'aide d'aucune cible (sauf pour identifier les clusters).  

In [ ]:
import torch; torch.manual_seed(0)
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.utils
import torch.distributions
import torchvision
import numpy as np
import matplotlib.pyplot as plt 
import matplotlib.cm as cm

In [ ]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir('drive/MyDrive/TP_ENM_2223')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_set = torchvision.datasets.MNIST('./data',
                 transform=torchvision.transforms.ToTensor(),
                 train=True,
                 download=True)

test_set = torchvision.datasets.MNIST('./data',
                 transform=torchvision.transforms.ToTensor(),
                 train=False,
                 download=True)

train_loader = DataLoader(train_set,
                         batch_size=128,
                         shuffle=True)

test_loader = DataLoader(test_set,
                         batch_size=128,
                         shuffle=True)

**Exercice 1** Un premier autoencodeur

Dans la suite, on considère un espace d'images $\mathcal{X}$, et un large échantillon d'images $x^{(i)}$ prises dans $\mathcal{X}$. 
Un auto-encodeur est un couple de deux réseaux de neurones $f_{\phi}, g_\theta$ 
que l'on entraîne simultanément à une tâche triviale: restituer le signal mis d'entrée. On cherche ainsi, le plus souvent, à minimiser :

$$ \mathcal{L}(\theta , \phi, X) =  \mathbb{E} {\bigg [}\: ||g_{\theta} ( f_{\phi} (X)) - X ||^2 {\bigg ]} $$

L'intérêt vient du fait que l'espace de sortie de $f_{\phi}$, qu'on appelle "espace latent", est de dimension réduite. Pour favoriser au mieux la reconstruction, $f_{\phi}$ doit "encoder" l'information utile dans l'espace latent, tandis que $g_{\theta}$ doit la décoder.
On peut ainsi espérer réduire la dimension d'un problème de partionnement.


**Q1** Les codes suivants permettent de construire des auto-encodeurs comprenant plus ou moins de couches cachées et un espace latent de dimension (*latent_dims*) plus ou moins grande. \
Vous pouvez étudier trois aspects:
- est-ce que l'apprentissage a bien convergé ? Y a-t-il surraprentissage ?
- est-ce que l'espace latent est organisé ? En particulier, voit-on des clusters ? Le K-Means est-il efficace dessus ? (La TSNE permettra de visualiser l'organisation de l'espace latent en grande dimension). \\
- peut-on se servir de $g_\theta$ comme un modèle génératif ?  


rq : https://fr.wikipedia.org/wiki/Courbe_de_Peano

In [ ]:
class ShallowEncoder(nn.Module):
    def __init__(self, latent_dims):
        super(ShallowEncoder, self).__init__()
        self.linear1 = nn.Linear(784, 512)
        self.linear2 = nn.Linear(512, latent_dims)

    def forward(self, x):
        x = torch.flatten(x, start_dim=1)
        x = F.relu(self.linear1(x))
        return self.linear2(x)

In [ ]:
class ShallowDecoder(nn.Module):
    def __init__(self, latent_dims):
        super(ShallowDecoder, self).__init__()
        self.linear1 = nn.Linear(latent_dims, 512)
        self.linear2 = nn.Linear(512, 784)

    def forward(self, z):
        z = F.relu(self.linear1(z))
        z = torch.sigmoid(self.linear2(z))
        return z.reshape((-1, 1, 28, 28))

In [ ]:
class ShallowAutoencoder(nn.Module):
    def __init__(self, latent_dims):
        super(ShallowAutoencoder, self).__init__()
        self.encoder = ShallowEncoder(latent_dims)
        self.decoder = ShallowDecoder(latent_dims)

    def forward(self, x):
        z = self.encoder(x)
        return self.decoder(z)

In [ ]:
def train(autoencoder, data, epochs=20):
    opt = torch.optim.Adam(autoencoder.parameters())
    for epoch in range(epochs):
        print(f"epoch : {epoch+1}/{epochs}")
        epoch_loss = 0.
        for i, (x, y) in enumerate(train_loader):
            x = x.to(device) # GPU
            opt.zero_grad()
            x_hat = autoencoder(x)
            loss = ((x - x_hat)**2).mean()
            loss.backward()
            opt.step()
            epoch_loss += loss.item()
        print(f"epoch loss : {epoch_loss:.3f}")
    return autoencoder

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim=784, inter_dims=[500,500,2000], latent_dims=10):
        super(Encoder,self).__init__()

        self.encoder=nn.Sequential(
            nn.Linear(input_dim, inter_dims[0]),
            nn.ReLU(True),
            nn.Linear(inter_dims[0], inter_dims[1]),
            nn.ReLU(True),
            nn.Linear(inter_dims[1], inter_dims[2]),
            nn.ReLU(True)
        )

        self.mu_l=nn.Linear(inter_dims[-1], latent_dims)

    def forward(self, x):
        x = torch.flatten(x, start_dim=1)
        e = self.encoder(x)

        mu = self.mu_l(e)

        return mu 


class Decoder(nn.Module):
    def __init__(self, input_dim=784, inter_dims=[500,500,2000], latent_dims=10):
        super(Decoder,self).__init__()

        self.decoder=nn.Sequential(
            nn.Linear(latent_dims, inter_dims[-1]),
            nn.ReLU(True),
            nn.Linear(inter_dims[-1], inter_dims[-2]),
            nn.ReLU(True),
            nn.Linear(inter_dims[-2], inter_dims[-3]),
            nn.ReLU(True),            
            nn.Linear(inter_dims[-3], input_dim),
            nn.Sigmoid()
        )
    def forward(self, z):
        x = self.decoder(z)
        return x.reshape((-1, 1, 28, 28))

class Autoencoder(nn.Module):
    def __init__(self, latent_dims):
        super(Autoencoder, self).__init__()
        self.encoder = Encoder(latent_dims=latent_dims)
        self.decoder = Decoder(latent_dims=latent_dims)

    def forward(self, x):
        z = self.encoder(x)
        return self.decoder(z)

In [ ]:
def plot_latent(autoencoder, data, num_batches=100):
    for i, (x, y) in enumerate(data):
        z = autoencoder.encoder(x.to(device))
        z = z.to('cpu').detach().numpy()
        plt.scatter(z[:, 0], z[:, 1], c=y, cmap='tab10', s=1)
        if i > num_batches:
            plt.colorbar()
            break


def plot_latent_3d(autoencoder, data, num_batches=5):
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')
    ax.set_xlabel('Z[0]')
    ax.set_ylabel('Z[1]')
    ax.set_zlabel('Z[2]')
    for i, (x, y) in enumerate(data):
        z = autoencoder.encoder(x.to(device))
        z = z.to('cpu').detach().numpy()
        d = ax.scatter(z[:, 0], z[:, 1], z[:, 2], c=y, cmap='tab10', s=2)

        if i > num_batches:
            fig.colorbar(d)
            break
    return ax


In [ ]:
def plot_reconstructed(autoencoder, r0=(-3, 3), r1=(-3, 3), n=12):
    w = 28
    img = np.zeros((n*w, n*w))
    for i, y in enumerate(np.linspace(*r1, n)):
        for j, x in enumerate(np.linspace(*r0, n)):
            z = torch.Tensor([[x, y, 0]]).to(device)
            x_hat = autoencoder.decoder(z)
            x_hat = x_hat.reshape(28, 28).to('cpu').detach().numpy()
            img[(n-1-i)*w:(n-1-i+1)*w, j*w:(j+1)*w] = x_hat
    plt.imshow(img, extent=[*r0, *r1])


Avec un espace latent de dimension 10:

In [ ]:
latent_dims = 10
# autoencoder = ShallowAutoencoder(latent_dims).to(device) 
autoencoder = Autoencoder(latent_dims).to(device) 
autoencoder = train(autoencoder, train_loader)

In [ ]:
def plot_reconstructed_d10(autoencoder, r0=(-3, 3), r1=(-3, 3), n=12):
    w = 28
    img = np.zeros((n*w, n*w))
    for i, y in enumerate(np.linspace(*r1, n)):
        for j, x in enumerate(np.linspace(*r0, n)):
            z = torch.Tensor([[x, y] + [0 for i in range(8)]]).to(device)
            x_hat = autoencoder.decoder(z)
            x_hat = x_hat.reshape(28, 28).to('cpu').detach().numpy()
            img[(n-1-i)*w:(n-1-i+1)*w, j*w:(j+1)*w] = x_hat
    plt.imshow(img, extent=[*r0, *r1])

**Q2** Lorsque l'espace latent est de taille plus grande, il n'est plus possible d'en cerner l'organisation. Un outil permet alors de plonger l'espace latent dans le plan en conservant la relation de proximité: c'est la [t-SNE](https://distill.pub/2016/misread-tsne/).
L'utiliser pour évaluer l'organisation en clusters dans l'espace latent.

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
def plotTSNE(model, dataset, size=100, perplexity=30, n_iter=1000):
  model.eval() 
  z = np.array([])
  first = True
  dataLoader = DataLoader(dataset, batch_size=128, shuffle=False,
                          num_workers=2)    
  it=iter(dataLoader)

  if size is None:
      size=len(dataLoader)

  # Calcul des vecteurs latents z
  nbreImgCount=0
  while(1):
          try :
              tmp=next(it)
          except:
              break    
          
          labels_batch = tmp[1]
          imgs = tmp[0]
          
          imgs = imgs.cuda()
          z_batch = model.encoder(imgs)  # plong. dans l'espace latent
          # z_batch = torch.flatten(imgs, start_dim = 1) # si l'on veut tsne brute
          z_batch = z_batch.cpu().detach().numpy()
          
          if len(z) == 0:
              z = z_batch
              labels = labels_batch

          else:
              z = np.concatenate((z,z_batch),axis=0)
              labels = np.concatenate((labels,labels_batch))
              
              
          nbreImgCount += len(imgs)
          print(nbreImgCount)
          if nbreImgCount >=size:
              break

  print('Dim vecteurs espace latent : {}',z.shape[1])

  # Calcul de la TSNE en 2D

  X2 = TSNE(n_components=2,
            verbose=2,
            init='pca',
            n_iter=n_iter,
            perplexity=perplexity).fit_transform(z)

  b=((labels-labels.min())/(labels.max()-labels.min())*255.).astype(int)
  c1 = cm.jet(b)

  fig=plt.figure('TSNE 2D ')

  for i,label in enumerate(labels):
      plt.text(X2[i, 0], X2[i, 1],str(label),color=c1[i],fontsize=12)
      
  plt.xlabel('X')
  plt.ylabel('Y')
  plt.axis([-50,50,-50,50])     

In [ ]:
model = autoencoder 
dataset = test_set  
size = 1000
n_iter = 1000
perplexity = 20
size= 1000

plt.rcParams['figure.dpi'] = 200
plotTSNE(model, dataset, size=1000, perplexity=perplexity, n_iter=n_iter)



**Q3** Pour évaluer le partionnement par K-moyennes sur l'esapce latent, on peut exploiter la matrice de confusion. Moyennant une juste assignation des classes à chacun des clusters, on peut calculer une justesse. La déterminer à partir des codes ci-dessous.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.mixture import GaussianMixture
  
Z = []
Y = []
with torch.no_grad():
    for x, y in test_loader:
        x = x.cuda()
        z1 = autoencoder.encoder(x)
        Z.append(z1)
        Y.append(y)

Z = torch.cat(Z, 0).detach().cpu().numpy()
Y = torch.cat(Y, 0).detach().numpy()

pre = KMeans(n_clusters=10).fit_predict(Z)
gmm = GaussianMixture(n_components=10, covariance_type='diag')
pre2 = gmm.fit_predict(Z)


plt.figure('matrice confusion kmeans')
c = confusion_matrix(Y, pre)
plt.imshow(c)
plt.colorbar()

Pour [trouver la permutation](https://en.wikipedia.org/wiki/Assignment_problem) qui maximise la justesse: le module linear_assignment, mis à disposition dans le drive. Ainsi chaque cluster est assigné à une classe.

In [ ]:
from linear_assignment_ import linear_assignment

In [ ]:
def cluster_acc(Y_pred, Y):
    D = max(Y_pred.max(), Y.max())+1
    w = np.zeros((D,D), dtype=np.int64)
    for i in range(Y_pred.size):
        w[Y_pred[i], Y[i]] += 1
    ind = linear_assignment(w.max() - w)
    return sum([w[i,j] for i,j in ind])/Y_pred.size, w

**Conclusion**: 

Avec un outillage standard (perceptrons multicouches + optim ADAM) les auto-encodeurs convergent sans difficulté.
L'espace latent d'un autoencodeur est organisé, les populations associées à chaque chiffre y forment des clusters plus ou moins nets.\
La séparation avec la TSNE est manifeste (nettement plus qu'avec les données brutes), mais un partitionnement sur l'espace latent avec des méthodes traditionnelles n'est pas très efficace: la justesse après réassignation n'atteint que 70% (certes, avec un nombre limité d'époques et sans chercher à tuner le réseau).

Néanmoins, ce chiffre est déjà remarquable. Noter que nous n'avons pas utilisé une seule cible pendant la phase d'entraînement. 

**Exercice 2** Un autoencodeur variationnel

Pour améliorer les propriétés de l'espace latent, l'autoencodeur "variationnel" 
a donné des résultats prometteurs.
Dans cet exercice, on décrit une version simple de ce modèle en termes pratiques. Mais il faut garder à l'esprit que ce modèle découle d'un cadre conceptuel profond. 

On pourra consulter les articles suivants avant de lire le code suivant.
- [un post](https://towardsdatascience.com/understanding-variational-autoencoders-vaes-f70510919f73) pour un public plus branché "info"
- [l'article "séminal"](https://arxiv.org/pdf/1312.6114v10.pdf) de D.Kingma (se concentrer sur l'intro).



In [ ]:
#Un premier VAE avec quelques couches complètement connectées

class ShallowVariationalEncoder(nn.Module):
    def __init__(self, latent_dims):
        super(ShallowVariationalEncoder, self).__init__()
        self.linear1 = nn.Linear(784, 512)
        self.linear2 = nn.Linear(512, latent_dims)
        self.linear3 = nn.Linear(512, latent_dims)

        self.N = torch.distributions.Normal(0, 1)
        self.N.loc = self.N.loc.cuda() 
        self.N.scale = self.N.scale.cuda()
        self.kl = 0

    def forward(self, x):
        x = torch.flatten(x, start_dim=1)
        x = F.relu(self.linear1(x))
        mu =  self.linear2(x)
        sigma = torch.exp(self.linear3(x))
        z = mu + sigma*self.N.sample(mu.shape)
        #Div de KL entre Nz et N01:
        self.kl = (sigma**2 + mu**2 - torch.log(sigma) - 1/2).sum()
        return z

class ShallowVariationalAutoencoder(nn.Module):
    def __init__(self, latent_dims):
        super(ShallowVariationalAutoencoder, self).__init__()
        self.encoder = ShallowVariationalEncoder(latent_dims)
        self.decoder = ShallowDecoder(latent_dims)

    def forward(self, x):
        z = self.encoder(x)
        return self.decoder(z)


**Q1** Quelles sont les différences avec l'AE classique ?
Quels rôles jouent les termes $\mu$, $\sigma$ et l'attribut *self.kl* ?

**Q2** Reprendre la fonction *train* et l'adapter à l'autoencodeur de façon à forcer la distribution sur la couche latente à suivre une loi normale centrée réduite.

In [ ]:
# Boucle d'apprentissage
def train(autoencoder, data, epochs=20):
    opt = torch.optim.Adam(autoencoder.parameters())
    for epoch in range(epochs):
        print(f"epoch : {epoch+1}/{epochs}")
        epoch_loss = 0.
        for x, y in data:
            x = x.to(device)
            opt.zero_grad()
            x_hat = autoencoder(x)
            # C'est la fonction de coût "ELBO" (très populaire depuis [Kingma14])
            loss = ((x - x_hat)**2).sum() + autoencoder.encoder.kl
            loss.backward()
            opt.step()
            epoch_loss += loss.item()
        print(f"epoch loss : {epoch_loss/len(data):.2f}")
    return autoencoder


**Q3** Visualiser les quelques résultats, la répartition des images dans l'espace latent en 2/10 dimensions. Evaluer le réseau en matière de partionnement.

In [ ]:
def plot_reconstructed_vae(autoencoder, r0=(-3, 3), r1=(-3, 3), n=12):
    w = 28
    img = np.zeros((n*w, n*w))
    for i, y in enumerate(np.linspace(*r1, n)):
        for j, x in enumerate(np.linspace(*r0, n)):
            z = torch.Tensor([[x, y]]).to(device)
            x_hat = autoencoder.decoder(z)
            x_hat = x_hat.reshape(28, 28).to('cpu').detach().numpy()
            img[(n-1-i)*w:(n-1-i+1)*w, j*w:(j+1)*w] = x_hat
    plt.imshow(img, extent=[*r0, *r1])

**Q4** Les VAE sont surtout connus pour être concurrents du GAN. En quoi est-ce une approche générative ? Illustrer.

**Exercice 3** 
Cet exercice présente une version du VAE adaptée à un clustering sur couches latente qui faisait [l'état de l'art en 2017](https://www.ijcai.org/proceedings/2017/0273.pdf).

La méthode consiste simplement à forcer la proximité avec un [modèle de mélange gaussien](https://fr.wikipedia.org/wiki/Mod%C3%A8le_de_m%C3%A9lange_gaussien).

Ici, l'énoncé est encore plus ouvert : faire tourner le modèle et évaluer ses performances en matière de clustering, de classification et de génération d'images.

In [ ]:
import torch
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader,TensorDataset
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir('drive/MyDrive/TP_dev')
from linear_assignment_ import linear_assignment

In [ ]:
train=MNIST(root=data_dir,train=True,download=True)
test=MNIST(root=data_dir,train=False,download=True)

X=torch.cat([train.data.float().view(-1,784)/255.,test.data.float().view(-1,784)/255.],0)
Y=torch.cat([train.targets,test.targets],0)

In [ ]:
def get_mnist(data_dir='./data/mnist/',batch_size=128):
    train=MNIST(root=data_dir,train=True,download=True)
    test=MNIST(root=data_dir,train=False,download=True)

    X=torch.cat([train.data.float().view(-1,784)/255.,test.data.float().view(-1,784)/255.],0)
    Y=torch.cat([train.targets,test.targets],0)

    dataset=dict()
    dataset['X']=X
    dataset['Y']=Y

    dataloader=DataLoader(TensorDataset(X,Y),batch_size=batch_size,shuffle=True,num_workers=4)

    return dataloader,dataset

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from torch.optim import Adam
import itertools
from sklearn.mixture import GaussianMixture
from sklearn.metrics import accuracy_score
import numpy as np
import os


def cluster_acc(Y_pred, Y):
    # from sklearn.utils.linear_assignment_ import linear_assignment
    assert Y_pred.size == Y.size
    D = max(Y_pred.max(), Y.max())+1
    w = np.zeros((D,D), dtype=np.int64)
    for i in range(Y_pred.size):
        w[Y_pred[i], Y[i]] += 1
    ind = linear_assignment(w.max() - w)
    return sum([w[i,j] for i,j in ind])*1.0/Y_pred.size, w


def block(in_c,out_c):
    layers=[
        nn.Linear(in_c,out_c),
        nn.ReLU(True)
    ]
    return layers


class Encoder(nn.Module):
    def __init__(self, input_dim=784, inter_dims=[500,500,2000], latent_dims=10):
        super(Encoder,self).__init__()

        self.encoder=nn.Sequential(
            nn.Linear(input_dim, inter_dims[0]),
            nn.ReLU(True),
            nn.Linear(inter_dims[0], inter_dims[1]),
            nn.ReLU(True),
            nn.Linear(inter_dims[1], inter_dims[2]),
            nn.ReLU(True)
        )

        self.mu_l=nn.Linear(inter_dims[-1], latent_dims)
        self.log_sigma2_l=nn.Linear(inter_dims[-1],hid_dim)

    def forward(self, x):
        # x = torch.flatten(x, start_dim=1)
        e = self.encoder(x)

        mu = self.mu_l(e)
        log_sigma2=self.log_sigma2_l(e)

        return mu, log_sigma2


class Decoder(nn.Module):
    def __init__(self, input_dim=784, inter_dims=[500,500,2000], latent_dims=10):
        super(Decoder,self).__init__()

        self.decoder=nn.Sequential(
            nn.Linear(latent_dims, inter_dims[-1]),
            nn.ReLU(True),
            nn.Linear(inter_dims[-1], inter_dims[-2]),
            nn.ReLU(True),
            nn.Linear(inter_dims[-2], inter_dims[-3]),
            nn.ReLU(True),            
            nn.Linear(inter_dims[-3], input_dim),
            nn.Sigmoid()
        )
    def forward(self, z):
        x = self.decoder(z)
        return x #.reshape((-1, 1, 28, 28))

class Autoencoder(nn.Module):
    def __init__(self, latent_dims):
        super(Autoencoder, self).__init__()
        self.encoder = Encoder(latent_dims=latent_dims)
        self.decoder = Decoder(latent_dims=latent_dims)

    def forward(self, x):
        z = self.encoder(x)
        return self.decoder(z)




class VaDE(nn.Module):
    def __init__(self,args):
        super(VaDE,self).__init__()
        self.encoder=Encoder()
        self.decoder=Decoder()

        self.pi_=nn.Parameter(torch.FloatTensor(args.nClusters,).fill_(1)/args.nClusters,requires_grad=True)
        self.mu_c=nn.Parameter(torch.FloatTensor(args.nClusters,args.hid_dim).fill_(0),requires_grad=True)
        self.log_sigma2_c=nn.Parameter(torch.FloatTensor(args.nClusters,args.hid_dim).fill_(0),requires_grad=True)

        self.args=args


    def pre_train(self, dataloader, pre_epoch=10):

        if  not os.path.exists('./pretrain_model.pk'):

          Loss=nn.MSELoss()
          opti=Adam(itertools.chain(self.encoder.parameters(),self.decoder.parameters()))

          print('Pretraining......')
          epoch_bar=tqdm(range(pre_epoch))
          for _ in epoch_bar:
              L=0
              for x,y in dataloader:
                  if self.args.cuda:
                      x = x.cuda()

                  z, _ = self.encoder(x)
                  x_ = self.decoder(z)
                  loss = Loss(x,x_)

                  L += loss.detach().cpu().numpy()

                  opti.zero_grad()
                  loss.backward()
                  opti.step()

              epoch_bar.write('L2={:.4f}'.format(L/len(dataloader)))

          self.encoder.log_sigma2_l.load_state_dict(self.encoder.mu_l.state_dict())

          Z = []
          Y = []
          with torch.no_grad():
              for x, y in dataloader:
                  if self.args.cuda:
                      x = x.cuda()

                  z1, z2 = self.encoder(x)
                  assert F.mse_loss(z1, z2) == 0
                  Z.append(z1)
                  Y.append(y)

          Z = torch.cat(Z, 0).detach().cpu().numpy()
          Y = torch.cat(Y, 0).detach().numpy()

          gmm = GaussianMixture(n_components=self.args.nClusters, covariance_type='diag')

          pre = gmm.fit_predict(Z)
          print('Acc={:.4f}%'.format(cluster_acc(pre, Y)[0] * 100))

          self.pi_.data = torch.from_numpy(gmm.weights_).cuda().float()
          self.mu_c.data = torch.from_numpy(gmm.means_).cuda().float()
          self.log_sigma2_c.data = torch.log(torch.from_numpy(gmm.covariances_).cuda().float())

          torch.save(self.state_dict(), './pretrain_model.pk')

        else:
            self.load_state_dict(torch.load('./pretrain_model.pk'))


    def predict(self,x):
        z_mu, z_sigma2_log = self.encoder(x)
        z = torch.randn_like(z_mu) * torch.exp(z_sigma2_log / 2) + z_mu
        pi = self.pi_
        log_sigma2_c = self.log_sigma2_c
        mu_c = self.mu_c
        yita_c = torch.exp(torch.log(pi.unsqueeze(0))+self.gaussian_pdfs_log(z,mu_c,log_sigma2_c))

        yita=yita_c.detach().cpu().numpy()
        return np.argmax(yita,axis=1)


    def ELBO_Loss(self,x,L=1):
        det=1e-10

        L_rec=0

        z_mu, z_sigma2_log = self.encoder(x)
        for l in range(L):

            z=torch.randn_like(z_mu)*torch.exp(z_sigma2_log/2)+z_mu

            x_pro=self.decoder(z)

            L_rec+=F.binary_cross_entropy(x_pro,x)

        L_rec/=L

        Loss=L_rec*x.size(1)

        pi=self.pi_
        log_sigma2_c=self.log_sigma2_c
        mu_c=self.mu_c

        z = torch.randn_like(z_mu) * torch.exp(z_sigma2_log / 2) + z_mu
        yita_c=torch.exp(torch.log(pi.unsqueeze(0))+self.gaussian_pdfs_log(z,mu_c,log_sigma2_c))+det

        yita_c=yita_c/(yita_c.sum(1).view(-1,1))#batch_size*Clusters

        Loss+=0.5*torch.mean(torch.sum(yita_c*torch.sum(log_sigma2_c.unsqueeze(0)+
                                                torch.exp(z_sigma2_log.unsqueeze(1)-log_sigma2_c.unsqueeze(0))+
                                                (z_mu.unsqueeze(1)-mu_c.unsqueeze(0)).pow(2)/torch.exp(log_sigma2_c.unsqueeze(0)),2),1))

        Loss-=torch.mean(torch.sum(yita_c*torch.log(pi.unsqueeze(0)/(yita_c)),1))+0.5*torch.mean(torch.sum(1+z_sigma2_log,1))

        return Loss

    def gaussian_pdfs_log(self,x,mus,log_sigma2s):
        G=[]
        for c in range(self.args.nClusters):
            G.append(self.gaussian_pdf_log(x,mus[c:c+1,:],log_sigma2s[c:c+1,:]).view(-1,1))
        return torch.cat(G,1)


    @staticmethod
    def gaussian_pdf_log(x,mu,log_sigma2):
        return -0.5*(torch.sum(np.log(np.pi*2)+log_sigma2+(x-mu).pow(2)/torch.exp(log_sigma2),1))

In [ ]:
description='VaDE'
datadir='./mnist'
batch_size = 800
nClusters=10
hid_dim=10
cuda=True


class Args:

    def __init__(self, description, batch_size, datadir, nClusters, hid_dim,\
                 cuda):
        self.description = description
        self.batch_size = batch_size
        self.datadir = datadir
        self.nClusters = nClusters
        self.hid_dim = hid_dim
        self.cuda = cuda  

args = Args(description, batch_size, datadir, nClusters, hid_dim, cuda)

In [ ]:
!pip install tensorboardX

In [ ]:
from tqdm import tqdm
import numpy as np
from torch.optim import Adam
from sklearn.metrics import accuracy_score
from torch.optim.lr_scheduler import StepLR
from tensorboardX import SummaryWriter
from sklearn.manifold import TSNE
import torch.nn as nn


def cluster_acc(Y_pred, Y):
    # from sklearn.utils.linear_assignment_ import linear_assignment
    # from scipy.optimize import linear_sum_assignment as linear_assignment
    assert Y_pred.size == Y.size
    D = max(Y_pred.max(), Y.max())+1
    w = np.zeros((D,D), dtype=np.int64)
    for i in range(Y_pred.size):
        w[Y_pred[i], Y[i]] += 1
    ind = linear_assignment(w.max() - w)
    return sum([w[i,j] for i,j in ind])*1.0/Y_pred.size, w



DL, _ = get_mnist(datadir,batch_size)

vade=VaDE(args)
if cuda:
    vade=vade.cuda()
    #Pour aller plus vite encore : (split de l'archi)
    # vade=nn.DataParallel(vade,device_ids=range(1))

vade.pre_train(DL,pre_epoch=50)

In [ ]:
opti=Adam(vade.parameters(),lr=2e-3)
lr_s=StepLR(opti,step_size=10,gamma=0.95)

writer=SummaryWriter('./logs')
epoch_bar=tqdm(range(300))


for epoch in epoch_bar:

    lr_s.step()
    L=0
    for x,_ in DL:
        if cuda:
            x=x.cuda()

        loss=vade.ELBO_Loss(x)

        opti.zero_grad()
        loss.backward()
        opti.step()

        L+=loss.detach().cpu().numpy()


    pre=[]
    tru=[]

    with torch.no_grad():
        for x, y in DL:
            if cuda:
                x = x.cuda()

            tru.append(y.numpy())
            pre.append(vade.predict(x))


    tru=np.concatenate(tru,0)
    pre=np.concatenate(pre,0)


    writer.add_scalar('loss',L/len(DL),epoch)
    writer.add_scalar('acc',cluster_acc(pre,tru)[0]*100,epoch)
    writer.add_scalar('lr',lr_s.get_lr()[0],epoch)

    epoch_bar.write('Loss={:.4f},ACC={:.4f}%,LR={:.4f}'.format(L/len(DL),cluster_acc(pre,tru)[0]*100,lr_s.get_lr()[0]))

In [ ]:
def plotTSNE_vade(model, dataset, size=100, perplexity=30, n_iter=1000):
  model.eval() 
  z = np.array([])
  first = True
  dataLoader = DataLoader(dataset, batch_size=128, shuffle=False,
                          num_workers=2)    
  it=iter(dataLoader)

  if size is None:
      size=len(dataLoader)

  # Calcul des vecteurs latents z
  nbreImgCount=0
  while(1):
          try :
              tmp=next(it)
          except:
              break    
          
          labels_batch = tmp[1]
          imgs = tmp[0]
          
          imgs = imgs.cuda()
          z_batch, _ = model.encoder(imgs)  # plong. dans l'espace latent
          # z_batch = torch.flatten(imgs, start_dim = 1) # si l'on veut tsne brute
          z_batch = z_batch.cpu().detach().numpy()
          
          if len(z) == 0:
              z = z_batch
              labels = labels_batch

          else:
              z = np.concatenate((z,z_batch),axis=0)
              labels = np.concatenate((labels,labels_batch))
              
              
          nbreImgCount += len(imgs)
          print(nbreImgCount)
          if nbreImgCount >=size:
              break

  print('Dim vecteurs espace latent : {}',z.shape[1])

  # Calcul de la TSNE en 2D

  X2 = TSNE(n_components=2,
            verbose=2,
            init='pca',
            n_iter=n_iter,
            perplexity=perplexity).fit_transform(z)

  b=((labels-labels.min())/(labels.max()-labels.min())*255.).astype(int)
  c1 = cm.jet(b)

  fig=plt.figure('TSNE 2D ')
  #ax2 = fig.add_subplot(111)
  #ax2.scatter(X2[:, 0], X2[:, 1],s=5,c=c1,marker='1')
  for i,label in enumerate(labels):
      plt.text(X2[i, 0], X2[i, 1],str(label),color=c1[i],fontsize=12)
      
  plt.xlabel('X')
  plt.ylabel('Y')
  plt.axis([-50,50,-50,50])     

In [ ]:
model = vade
dataset = test_set  
size = 1000
n_iter = 1000
perplexity = 20
size= 1000

plt.rcParams['figure.dpi'] = 200
plotTSNE(model, dataset, size=1000, perplexity=perplexity, n_iter=n_iter)

In [ ]:
vade.encoder

**Conclusion** \
Avec cette dernière approche, on parvient à un partionnement presque parfait des données MNIST, sans aucune annonation manuelle. Ce type de méthode offre donc de nombreuses perspectives en matière de partionnement, de classification d'image, mais plus généralement de toute tâche où ré-entraîner un encodeur déjà appris peut laisser espérer une amélioration.\
Evidemment, les chiffres de MNIST ne présentent pas la même difficulté que des images ou des sons réels. Mais d'un autre côté, les méthodes présentées ici sont rudimentaires. On trouve par exemple, dans la littérature, un panel de [tâches "gratuites"](https://project.inria.fr/paiss/files/2018/07/zisserman-self-supervised.pdf) (ie, sans besoin d'annotation) plus sophistiquées que la simple reproduction de l'entrée à laquelle est entraîné l'auto-encodeur de base.

